# Notes


In [1]:
import pandas as pd
import columns
from fields import Fields

In [2]:
df = pd.read_parquet("./data/us_births.parquet", engine="pyarrow")

In [3]:
# df.describe()

In [4]:
df_grouped = df.groupby(str(Fields.CA_DOWN)).size().reset_index(name="counts")
df_grouped = df_grouped.set_index(str(Fields.CA_DOWN))
df_grouped

,counts
ca_down,
C,7728
P,10871
U,124596
Y,54


In [6]:
# group by DOB_YY sum DS_LB_CHANCE
df_grouped_sum = (
    df.groupby(str(Fields.DOB_YY))
    .agg(total_ds_lb_ns=pd.NamedAgg(column=str(Fields.DS_LB_CHANCE), aggfunc="sum"))
    .reset_index()
)
df_grouped_sum

,dob_yy,total_ds_lb_chance
0,0,0.038658
1,2004,7440.391929
2,2005,7493.838661
3,2006,7665.960631
4,2007,7748.428247
5,2008,7699.195333
6,2009,7565.147534
7,2010,7439.844521
8,2011,7426.414635
9,2012,7500.288505


In [13]:
df[str(Fields.DS_C)] = df[str(Fields.CA_DOWN)].apply(lambda x: 1 if x == "C" else 0)

df[str(Fields.DS_P)] = df[str(Fields.CA_DOWN)].apply(lambda x: 1 if x == "P" else 0)

df[str(Fields.DS_N)] = df[str(Fields.CA_DOWN)].apply(lambda x: 1 if x == "N" else 0)

df[str(Fields.DS_U)] = df[str(Fields.CA_DOWN)].apply(lambda x: 1 if x == "U" else 0)

df[str(Fields.DS_CORP)] = df[str(Fields.DS_C)] + df[str(Fields.DS_P)]

In [20]:
# group by DOB_YY and counts of each value of df[str(Fields.CA_DOWN)]
annual_counts_df = (
    df.groupby(str(Fields.DOB_YY))[str(Fields.CA_DOWN)]
    .value_counts()
    .unstack(fill_value=0)
    .reset_index()
)

annual_counts_df = annual_counts_df.set_index(str(Fields.DOB_YY))
annual_counts_df

ca_down,C,P,U,Y
dob_yy,,,,
0,0,0,0,54
2004,148,335,7260,0
2005,220,494,8592,0
2006,327,702,9634,0
2007,324,800,20941,0
2011,565,1087,28410,0
2018,921,1187,6107,0
2019,923,1107,5468,0
2020,894,1060,5165,0


In [21]:
# group by DOB_YY and sum of DS_LB_CHANCE
annual_ds_lb_df = (
    df.groupby(str(Fields.DOB_YY))
    .agg(total_ds_lb_ns=pd.NamedAgg(column=str(Fields.DS_LB_CHANCE), aggfunc="sum"))
    .reset_index()
)
annual_ds_lb_df = annual_ds_lb_df.set_index(str(Fields.DOB_YY))
annual_ds_lb_df

,total_ds_lb_ns
dob_yy,
0,0.038658
2004,7440.391929
2005,7493.838661
2006,7665.960631
2007,7748.428247
2008,7699.195333
2009,7565.147534
2010,7439.844521
2011,7426.414635


In [22]:
annual_ds_lb_counts = annual_counts_df.join(annual_ds_lb_df)
annual_ds_lb_counts

,C,P,U,Y,total_ds_lb_ns
dob_yy,,,,,
0,0,0,0,54,0.038658
2004,148,335,7260,0,7440.391929
2005,220,494,8592,0,7493.838661
2006,327,702,9634,0,7665.960631
2007,324,800,20941,0,7748.428247
2011,565,1087,28410,0,7426.414635
2018,921,1187,6107,0,8023.239877
2019,923,1107,5468,0,8076.697233
2020,894,1060,5165,0,7907.029413
